In [1]:
import race_results_scanner
import google_cloud_downloader as gcd
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import os
import header_scanner as hs


#pdf = "Error_PDFS/AQU_11_04_2022.pdf"
pdf = "Example_PDFS/equibaseFile.pdf"


In [2]:
#Testing for page

page_list = race_results_scanner.get_page_list(pdf)
target_page = 6

page_list_index = 0
for page in page_list:
    if(target_page == page['page_num']):
        break
    page_list_index += 1
    
reader = PdfReader(pdf) #File to be scanned
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()

page = page_list[page_list_index]
header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
for field,value in header.items():
    print(field ,':', value)
result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
print(result_tables[0])
print(result_tables[1])

The PDF <_io.BufferedReader name='Error_PDFS/AQU_11_04_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


track : Aqueduct
date : November 4, 2022
race_num : 6
criteria : FOR MAIDENS, FILLIES TWO YEARS OLD FOALED IN NEW YORK STATE AND APPROVED BY THE NEW YORK STATE-BRED REGISTRY
track_length :  Six Furlongs On The Dirt
purse : $70,000
weather : Cloudy
track_type : Fast
off_time : 2:49
start : Good for all
race_type : maiden special weight
Looking for tables...
Found table top bound at 134.12199999999996
Found table bottom bound at 235.42199999999988
Looking for tables...
Found table top bound at 411.933
Found table bottom bound at 506.567
    Last Raced   Pgm jockey_first_name  jockey_last_name  Start 1/4 1/2 Str  \
0  12Aug22 SAR   3.0            Javier        Castellano    1.0   2   1   1   
1   1Oct22 BAQ   4.0          Kendrick         Carmouche    4.0   3   3   2   
2   1Oct22 BAQ   9.0            Trevor          McCarthy    2.0   7   7   4   
3          ---   7.0        Jacqueline             Davis    7.0   4   4   3   
4   5Sep22 SAR   8.0            Hector         Diaz  Jr.    6.0 

In [3]:
result_tables[0]

,Last Raced,Pgm,jockey_first_name,jockey_last_name,Start,1/4,1/2,Str,Fin,Odds,Comments,Horse Name,Last Raced_super_script,1/4_length_behind,1/2_length_behind,Str_length_behind,Fin_length_behind,Wgt,M/E
0,12Aug22 SAR,3.0,Javier,Castellano,1.0,2,1,1,1,0.80*,"pressed 3w, inched",Highway Harmony,1 2,1 1/2,1/2,1,1 1/4,119,- - 2
1,1Oct22 BAQ,4.0,Kendrick,Carmouche,4.0,3,3,2,2,2.75,"4w upper, repelled",Vax,1 5,2 1/2,1 1/2,3,2,119,b 3
2,1Oct22 BAQ,9.0,Trevor,McCarthy,2.0,7,7,4,3,9.70,"chased 6-5w, mild ki",Clover Street,1 2,6,6 1/2,1,5 1/2,119,- - 7
3,---,7.0,Jacqueline,Davis,7.0,4,4,3,4,13.20,"5w upper, weakened",Fema Funds,NaN,Head,2,1/2,5 1/2,119,- - 5
4,5Sep22 SAR,8.0,Hector,Diaz Jr.,6.0,5,5,6,5,9.70,"chased 5-4w, tired",Lady Quinn,1 6,1,1,2,1 1/2,120,b 6
5,25Sep22 BAQ,2.0,Luis,Rodriguez Castro,3.0,1,2,5,6,42.25,"under pressure, tired",Blanchiment,1 2,1/2,Head,1 1/2,Nose,119,- - 1
6,17Sep22 BAQ,6.0,Jose,Gomez,5.0,6,6,7,7,34.25,"chased 3-4w, no imp",Bank Holiday,1 10,4,1,6,4 1/2,114,- - 4
7,---,10.0,Jorge,Vargas Jr.,8.0,8,8,8,8,12.10,trailed throughout,Britney Is Free,Fractional Times: 22.48,NaN,NaN,NaN,NaN,119,- - 8


In [2]:
#Testing for whole pdf
master_df = pd.DataFrame()
page_list = race_results_scanner.get_page_list(pdf)
print(page_list)
    
reader = PdfReader(pdf) #File to be scanned
number_of_pages = len(reader.pages) #Number of pages
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()
i = 0
for page in page_list:
    header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
    result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
                
    #Combine into page DF
    top_table = result_tables[0]
    bottom_table = result_tables[1]
    #Dropping horse name for merge
    bottom_table = bottom_table.drop("Horse Name", axis = 1)
    merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
    for field,value in header.items():
        merged_df[field] = value
        #Changing all cols to object type 
        merged_df = merged_df.astype(object)
        #For every page df
    #Merge into master pdf df
    if(len(pdf_df) < 1):
        pdf_df = merged_df
    else:
        pdf_df = pd.merge(pdf_df, merged_df, how = 'outer')
            
    print(pdf_df)
    #Merging into master 
    #if(len(master_df) < 1):
     #   master_df = pdf_df
    #else:
    #    master_df = pd.merge(master_df,pdf_df, how = 'outer')

The PDF <_io.BufferedReader name='Example_PDFS/equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 5}, {'page_num': 1, 'horse_count': 7}, {'page_num': 2, 'horse_count': 5}, {'page_num': 3, 'horse_count': 6}, {'page_num': 4, 'horse_count': 8}, {'page_num': 5, 'horse_count': 7}, {'page_num': 6, 'horse_count': 5}, {'page_num': 7, 'horse_count': 8}, {'page_num': 8, 'horse_count': 8}, {'page_num': 9, 'horse_count': 8}, {'page_num': 10, 'horse_count': 5}]
Looking for tables...
Found table top bound at 135.24599999999995
Found table bottom bound at 206.54599999999994
Looking for tables...


The PDF <_io.BufferedReader name='Example_PDFS/equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table top bound at 404.8310000000001
Found table bottom bound at 479.4650000000001
    Last Raced  Pgm jockey_first_name jockey_last_name   PP Start 1/4 3/8 Str  \
0   2Jul22 MTH  4.0             Albin          Jimenez  4.0   3.0   2   2   2   
1   2Jul22 MTH  6.0             Angel        Rodriguez  5.0   2.0   3   3   1   
2  30Jun22 DEL  1.0              Jose           Ferrer  1.0   5.0   5   5   4   
3  22Jul22 MTH  2.0             Jorge      Vargas  Jr.  2.0   4.0   4   4   5   
4  30Jul22 MTH  3.0             Jomar           Torres  3.0   1.0   1   1   3   

  Fin  ... race_num                                           criteria  \
0   1  ...        1  (PLUS UP TO 40% NJB) FOR TWO YEAR OLDS WHICH H...   
1   2  ...        1  (PLUS UP TO 40% NJB) FOR TWO YEAR OLDS WHICH H...   
2   3  ...        1  (PLUS UP TO 40% NJB) FOR TWO YEAR OLDS WHICH H...   
3   4  ...        1  (PLUS UP TO 40% NJB) FOR TWO YEAR OLDS WHICH H...   
4   5  ...        1  (PLUS UP TO 40% NJB) FOR TWO YEAR

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Example_PDFS/equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


...

Found table bottom bound at 506.567
     Last Raced  Pgm jockey_first_name jockey_last_name   PP Start  1/4  3/8  \
0    2Jul22 MTH  4.0             Albin          Jimenez  4.0   3.0    2    2   
1    2Jul22 MTH  6.0             Angel        Rodriguez  5.0   2.0    3    3   
2   30Jun22 DEL  1.0              Jose           Ferrer  1.0   5.0    5    5   
3   22Jul22 MTH  2.0             Jorge      Vargas  Jr.  2.0   4.0    4    4   
4   30Jul22 MTH  3.0             Jomar           Torres  3.0   1.0    1    1   
5    5Aug22 MTH  5.0             Albin          Jimenez  4.0   7.0    6  NaN   
6   15Aug22 MTH  4.0          Madeline          Rowland  3.0   6.0    7  NaN   
7    5Aug22 MTH  6.0             Jairo           Rendon  5.0   3.0    2  NaN   
8   17Jul22 MTH  3.0             Angel        Rodriguez  2.0   4.0    5  NaN   
9   30Jul22 MTH  8.0            Carlos            Rojas  7.0   5.0    4  NaN   
10  30Jul22 MTH  7.0             Manny          Jimenez  6.0   2.0    3  NaN   
11  

Looking for tables...
Found table top bound at 143.12199999999996
Found table bottom bound at 234.42199999999988
Looking for tables...
Found table top bound at 439.997


C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Example_PDFS/equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 542.567
     Last Raced  Pgm jockey_first_name  jockey_last_name   PP Start 1/4  3/8  \
0    2Jul22 MTH  4.0             Albin           Jimenez  4.0   3.0   2    2   
1    2Jul22 MTH  6.0             Angel         Rodriguez  5.0   2.0   3    3   
2   30Jun22 DEL  1.0              Jose            Ferrer  1.0   5.0   5    5   
3   22Jul22 MTH  2.0             Jorge       Vargas  Jr.  2.0   4.0   4    4   
4   30Jul22 MTH  3.0             Jomar            Torres  3.0   1.0   1    1   
..          ...  ...               ...               ...  ...   ...  ..  ...   
62   5Aug22 MTH  2.0             Manny           Jimenez  2.0   7.0   4  NaN   
63   9Jul22 MTH  7.0              Paco             Lopez  7.0   6.0   7  NaN   
64   1Aug22 MTH  5.0              Omar  Hernandez Moreno  5.0   4.0   2  NaN   
65  29Jul22 LRL  6.0             Jomar            Torres  6.0   8.0   8  NaN   
66  28Jul22 DEL  1.0               Nik            Juarez  1.0   3.0   5  NaN   

   

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [3]:
pdf_df

,Last Raced,Pgm,jockey_first_name,jockey_last_name,PP,Start,1/4,3/8,Str,Fin,...,1/2_length_behind,3/4_length_behind,1/2_RLP,3/4_RLP,1/2_length_behind_RLP,3/4_length_behind_RLP,3/16,3/16_length_behind,3/16_RLP,3/16_length_behind_RLP
0,2Jul22 MTH,4.0,Albin,Jimenez,4.0,3.0,2,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2Jul22 MTH,6.0,Angel,Rodriguez,5.0,2.0,3,3,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30Jun22 DEL,1.0,Jose,Ferrer,1.0,5.0,5,5,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22Jul22 MTH,2.0,Jorge,Vargas Jr.,2.0,4.0,4,4,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30Jul22 MTH,3.0,Jomar,Torres,3.0,1.0,1,1,3,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,8Aug22 MTH,5.0,Angel,Castillo,4.0,1.0,1,NaN,1,1,...,2 1/2,NaN,1,NaN,2 1/2,NaN,NaN,NaN,NaN,NaN
68,31Jul22 MTH,3.0,Paco,Lopez,2.0,2.0,2,NaN,2,2,...,2 1/2,NaN,3,NaN,2 1/2,NaN,NaN,NaN,NaN,NaN
69,29Jul22 MTH,1.0,Samy,Camacho,1.0,3.0,3,NaN,3,3,...,Head,NaN,2,NaN,2 1/2,NaN,NaN,NaN,NaN,NaN
70,6Aug22 MTH,4.0,Manny,Jimenez,3.0,4.0,4,NaN,4,4,...,4,NaN,4,NaN,5,NaN,NaN,NaN,NaN,NaN


In [6]:
pdf_df.to_csv("ARAP.csv")